In [157]:
using Plots
import Pkg; Pkg.add("DataFrames")
using DataFrames
using Statistics

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [156]:
function calc_angle(transduct_dis::Float64, diameter::Float64)
    theta_rad::Float64 = atan(diameter / transduct_dis)
    theta_degree::Float64 = rad2deg(theta_rad)
    L_dis::Float64 = sqrt(transduct_dis^2 + diameter^2)
    return theta_degree, L_dis
end

calc_angle (generic function with 1 method)

In [155]:
function calc_velocity(angle::Float64, L_dis::Float64, vel::Float64)
    c::Float64 = 1480.00
    td = L_dis / (c + vel * cosd(angle))
    tu = L_dis / (c - vel * cosd(angle))
    delta_time = tu - td
    v::Float64 = (L_dis * delta_time) / (2 * cosd(angle) * tu * td)
    return v
end

calc_velocity (generic function with 1 method)

In [100]:
function add_noise(value::Float64, noise_level::Float64)
    return value + (randn()*2-1) * noise_level
end

add_noise (generic function with 1 method)

In [154]:
# Função para calcular o perfil de velocidade parabólico
function parabolic_velocity_profile(r::Float64, R::Float64, vmax::Float64)
    return vmax * (1 - (r / R)^2)
end

# Função para calcular o tempo de trânsito usando o perfil parabólico
function calc_vel_parabolic(angle::Float64, L_dis::Float64, R::Float64, vmax::Float64)
    c = 1480.0
    num_points = 100
    rs = range(-R, stop=R, length=num_points)
    vs = parabolic_velocity_profile.(rs, R, vmax)
    avg_velocity = mean(vs)
    td = L_dis / (c + avg_velocity * cosd(angle))
    tu = L_dis / (c - avg_velocity * cosd(angle))
    delta_time = tu - td
    v::Float64 = (L_dis * delta_time) / (2 * cosd(angle) * tu * td)
    return [v]
end

calc_vel_parabolic (generic function with 1 method)

In [153]:
function calc_MAE(v::Vector{Float64}, v_hat::Vector{Float64})
    return mean(abs.(v .- v_hat))
end

calc_MAE (generic function with 1 method)

In [171]:
len = 10
velocitys = collect(range(0.1, 10.0, length=len))  # Velocities in m/s
distance_btw_transductores = collect(range(0.1, 50, length=len))  # cm
diameters = collect(range(0.1, 50.0, length=len))  # cm


true_velocities = Float64[]
noisy_velocities = Float64[]
errors = Float64[]
parameter_combinations = []


for velocity in velocitys
    for distance in distance_btw_transductores
        for diameter in diameters
            angle, L_dis = calc_angle(distance, diameter)
            true_velocity = calc_velocity(angle, L_dis, velocity)
            push!(true_velocities, true_velocity)
            noisy_velocity = add_noise(true_velocity, 0.01)
            push!(noisy_velocities, noisy_velocity)
            error = calc_MAE([true_velocity], [noisy_velocity])
            push!(errors, error)
            push!(parameter_combinations, (velocity, distance, diameter))
        end
    end
end

